### hive

```
将结构化数据映射为一张数据库表
结构化数据："规整的数据"，类似于mysql的二维表
提供HQL(类似于SQL)查询功能
底层数据是存储在 HDFS
将SQL语句转换为MapReduce任务运行

元数据存储在mysql中

hive和mysql的对比
hive：
只支持插入操作
执行延迟高
不支持事务
处理数据规模大
可扩展性强

数据类型：
简单数据类型：大部分和mysql类似
复杂数据类型：array，map，struct

表类型：内部表和外部表

hive没有专门的数据存储格式
在创建表时指定数据中的分隔符
数据模型：
db：在 hdfs 中表现为 hive.metastore.warehouse.dir(默认为：hdfs:///user/hive/warehourse/*.db) 目录下一个文件夹
table：在 hdfs 中表现所属 db 目录下一个文件夹
partition(分区)：在 hdfs 中表现为 table 目录下的子目录

hive的启动流程：
1，启动hdfs
2，启动yarn
3，启动docker
4，启动mysql
5，启动hive的元数据服务
    hive --service metastore&
6，启动hive

内部表和外部表：
数据管理情况
删除元数据是否连带删除数据
数据存放位置

```

### flume：日志收集工具

```
使用版本：flume-ng

三个基本结构：
source：从哪里拿数据(数据源)
channel：临时的缓存
sink：放到哪里去(数据放置位置)

外部结构：agent
一个agent包含三部分：source、channel、sink

flume数据传输的基本单元：event
event(byte array)由两部分组成
event header：
event body：
收集文本文件，一个event代表一行数据

#定义agent及其三个子组件的名字
#agent的名字是：a1
#agent的source是r1
#agent的channel：c1
#agent的sink：k1
a1.sources = r1
a1.sinks = k1
a1.channels = c1

#对agent的source进行详细配置
a1.sources.r1.type = netcat
a1.sources.r1.bind = 192.168.19.137
a1.sources.r1.port = 44444

#对agent的sink进行详细配置
a1.sinks.k1.type = logger

#对agent的channel进行详细配置
a1.channels.c1.type = memory
a1.channels.c1.capacity = 1000
a1.channels.c1.transactionCapacity = 100

#配置source和channel的关系及其sink和channel的关系
a1.sources.r1.channels = c1
a1.sinks.k1.channel = c1

netcat_memory_logger

启动flume：/root/bigdata/flume/bin/flume-ng agent -n a1 -f test.conf

作业：
从A服务器采集日志文件access.log、nginx.log到C服务器，
从B服务器采集目录中新增的文件到C服务器，
C服务器的数据最终存储到hdfs。
要求：从access.log中获取的数据最终存储到hdfs的access目录中，从nginx.log中获取的数据最终存储到hdfs的nginx目录中，从B服务器中目录采集的数据最终存储到hdfs的spool目录中
```

with
tmp_detail as
(
    select
        user_id,
        sku_id, 
        sum(sku_num) sku_num,   
        count(*) order_count, 
        sum(od.order_price*sku_num) order_amount
    from dwd_order_detail od
    where od.dt='2019-02-11'
    group by user_id, sku_id
)  

insert overwrite table dws_sale_detail_daycount partition(dt='2019-02-11')

select 
    tmp_detail.user_id,
    tmp_detail.sku_id,
    u.gender,
    months_between('2019-02-11', u.birthday)/12  age, 
    u.user_level,
    price,
    sku_name,
    tm_id,
    category3_id,
    category2_id,
    category1_id,
    category3_name,
    category2_name,
    category1_name,
    spu_id,
    tmp_detail.sku_num,
    tmp_detail.order_count,
    tmp_detail.order_amount 
from tmp_detail 
left join dwd_user_info u on tmp_detail.user_id =u.id and u.dt='2019-02-11'
left join dwd_sku_info s on tmp_detail.sku_id =s.id and s.dt='2019-02-11'
;




export JAVA_HOME=/root/bigdata/jdk
export HBASE_MANAGES_ZK=true  --如果你是使用hbase自带的zk就是true，如果使用自己的zk就是false


